# Extract and Aggregate Pedestrian Data for Historical Pedestrian Dashboard
### Hannah Catri 217569841
This workbook extracts the monthly pedestrian data directly from Melbourne Open data and aggregates it, and exports it to a csv to make it more usable for the Tableau visualisations found here https://public.tableau.com/profile/hannah.catri4817#!/vizhome/CityOfMelbournePedestrianData_HistoricalData/CityofMelbourneHistoricalPedestrianDataTrends?publish=yes


In [123]:
# Import packaged
import pandas as pd
from sodapy import Socrata

In [69]:
# Get the number of rows in the dataset
# Save as a variable in case paging method of data retireval is needed
num=pd.read_json("https://data.melbourne.vic.gov.au/resource/b2ak-trbp.json?$select=COUNT(date_time)")
maxNumber=num.iloc[0,0]
print(maxNumber)

3482938


In [66]:
# Configure the Socrata client and create the generator that will retrieve the data
client = Socrata("data.melbourne.vic.gov.au", None)
dataset_gen=client.get_all("b2ak-trbp")

In [68]:
# Extract the dataset, this may take a while.
# Use shape to confirm the size matches the rows as per above maxNumber.
results_df = pd.DataFrame.from_records(dataset_gen)
results_df.shape

(3482938, 10)

In [70]:
# Preview dataset 
results_df.head()

,id,date_time,year,month,mdate,day,time,sensor_id,sensor_name,hourly_counts
0,2887628,2019-11-01T17:00:00.000,2019,November,1,Friday,17,34,Flinders St-Spark La,300
1,2887629,2019-11-01T17:00:00.000,2019,November,1,Friday,17,39,Alfred Place,604
2,2887630,2019-11-01T17:00:00.000,2019,November,1,Friday,17,37,Lygon St (East),216
3,2887631,2019-11-01T17:00:00.000,2019,November,1,Friday,17,40,Lonsdale St-Spring St (West),627
4,2887632,2019-11-01T17:00:00.000,2019,November,1,Friday,17,36,Queen St (West),774


In [83]:
#Reformat data types for grouping and analysis
#change the 'date_time' field to actual date variable
results_df['date_time'] = pd.to_datetime(results_df['date_time'])
#change hourly counts to int
results_df['hourly_counts']=results_df.hourly_counts.astype(int)
# confirm results
results_df.dtypes

id                       object
date_time        datetime64[ns]
year                     object
month                    object
mdate                    object
day                      object
time                     object
sensor_id                object
sensor_name              object
hourly_counts            object
dtype: object

In [119]:
#create by day/hour aggregates
grouped_by_dayTime = pd.DataFrame(results_df.groupby('date_time').hourly_counts.sum())
grouped_by_dayTime.head()

,hourly_counts
date_time,
2009-05-01 00:00:00,952
2009-05-01 01:00:00,510
2009-05-01 02:00:00,222
2009-05-01 03:00:00,188
2009-05-01 04:00:00,294


In [121]:
#export file to .csv to be picked up Tableau workbook
grouped_by_dayTime.to_csv('grouped_by_dayTime.csv', encoding='utf-8')